In [90]:
from qiskit import *
from qiskit.visualization import plot_bloch_multivector, plot_histogram, array_to_latex
import itertools
import numpy as np
from time import sleep
import pandas as pd
import progressbar
import csv
import ast
sim = Aer.get_backend('qasm_simulator')
%matplotlib inline

## Find all permutations and synonyms

In [91]:
gates = ['x', 'y', 'z', 'h']
gateCombos = []
comboSV = []
similar = []

In [92]:
#Find all permutations and append their statevectors and permutations to lists
for L in range(1, len(gates)+1):
    
    #for subset in itertools.permutations(gates, L):
    for subset in itertools.product(gates, repeat=L):
        #print(subset)
        gateCombos.append(subset)
#Add a no gate option
gateCombos.append('')
        
#find statevectors of each possible permutation and store in list
for i in range(len(gateCombos)):
    qc = QuantumCircuit(1, 1) #1 quantum, 1 classical
    #print(gateCombos[i])
    
    for j in gateCombos[i]:
        getattr(qc , j)(0)
        #print(qc)
    # Let's get the result:
    qc.measure_all()
    result = execute(qc, backend = sim, shots=1024).result()
    counts = result.get_counts()
    lst = list(counts.items())
    temp = []
    for j in range(len(lst)):
        temp.append(lst[j][0])
    comboSV.append(temp)
    # print(final_state)


In [93]:
len(gateCombos)

341

In [94]:
len(comboSV)

341

In [95]:
#Create a list of synonym pairs based on if statevectors are equal
bar = progressbar.ProgressBar(maxval=len(comboSV)).start()
for i in range(len(comboSV)):
    for j in range(len(comboSV)):
        temp = []
        if(i != j and comboSV[i]==comboSV[j]):
            temp.append(gateCombos[i])
            temp.append(gateCombos[j])
            # print('')
            # print(comboSV[i])
            # print(comboSV[j])
            # print(temp)
            similar.append(temp)
    bar.update(i)
            

In [97]:
len(similar)

29028

In [98]:
bar = progressbar.ProgressBar(maxval=len(similar)).start()
# provider = IBMQ.get_provider('ibm-q')
# backend = provider.get_backend('ibmq_armonk')
#sort largest synonym in pair first
#If synonyms are the same in length, check computing times of each and set slower one as target and quicker one as the replacement
listToRemove = []
for i in range(len(similar)):
    if len(similar[i][1]) > len(similar[i][0]):
        similar[i].insert(0, similar[i].pop(1))
    elif len(similar[i][1]) == len(similar[i][0]):
        #1
        qc = QuantumCircuit(1, 1) #1 quantum, 1 classical
        for j in similar[i][0]:
            getattr(qc , j)(0)
        qc.measure_all()
        result = execute(qc, backend=sim, shots = 1000).result()
        time1 = result.time_taken
        #2
        qc = QuantumCircuit(1, 1) #1 quantum, 1 classical
        for j in similar[i][1]:
            getattr(qc , j)(0)
        qc.measure_all()
        result = execute(qc, backend=sim, shots = 1000).result()
        time2 = result.time_taken
        if time2 > time1:
            similar[i].insert(0, similar[i].pop(1))
        if time2 == time1:
            listToRemove.append(i)
    bar.update(i)
listToRemove.sort(reverse=True)
print("Removing Elements at Indices: " + str(listToRemove))
listToRemove.sort(reverse=True)
for r in listToRemove:
    similar.pop(r)

Removing Elements at Indices: [28484, 28483, 27940, 27929, 27852, 27062, 26438, 26253, 26199, 25028, 23543, 23291, 23264, 23162, 23022, 22771, 22435, 22394, 22235, 20284, 19839, 19481, 18963, 17594, 17126, 16922, 16810, 16742, 16686, 16648, 16644, 16587, 16421, 16379, 16364, 16339, 16293, 16280, 16248, 16167, 16089, 16021, 15903, 15744, 15738, 15711, 15674, 15655, 15564, 15267, 15196, 15124, 14898, 14887, 14858, 14857, 14800, 14760, 14556, 14538, 14425, 14331, 14292, 14288, 14271, 14091, 14050, 14043, 13683, 13659, 13425, 12674, 12633, 12599, 12534, 12521, 12517, 12281, 12261, 11944, 11885, 11840, 11765, 11760, 11623, 11621, 11603, 11596, 11595, 11594, 11584, 11568, 11533, 11531, 11513, 11486, 11437, 11431, 11427, 11414, 11405, 11363, 11358, 11262, 11216, 11195, 11192, 11179, 11177, 11174, 11065, 11062, 10834, 10802, 10799, 10732, 10711, 10704, 10647, 10629, 10528, 10476, 10443, 10384, 10351, 10333, 10264, 10244, 10197, 10171, 10161, 10100, 10015, 10009, 9995, 9936, 9901, 9895, 9756, 9

In [99]:
len(similar)

28808

## Create Database File

In [100]:
with open(r"key.csv", 'w', newline='', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(['Target', 'Replacement'])
    writer.writerows(similar)



### CSV to Pandas DF

In [101]:
df = pd.read_csv(r"key.csv")

### Filtering out multiple replacements for one target

In [102]:
unique = df['Target'].unique()
unique = unique.tolist()
targets = df['Target'].tolist()

In [103]:
bar = progressbar.ProgressBar(maxval=len(unique)).start()
remove = []
for i in range(len(unique)):
    bar.update(i)
    times = []
    indices = []
    for j in range(len(targets)):
        if unique[i] == targets[j]:
            indices.append(j)
    for x in range(len(indices)):
        qc = QuantumCircuit(1, 1) #1 quantum, 1 classical
        for q in similar[x][1]:
            getattr(qc , q)(0)
        qc.measure(0, 0)
        result = execute(qc, backend=sim, shots = 1000).result()
        times.append(result.time_taken)
    least = min(times)
    for e in indices:
        remove.append(e)
    remove.remove(indices[times.index(least)])
remove.sort(reverse=True)
#print(remove)
for i in remove:
    similar.pop(i)

In [15]:
len(similar)

328

In [104]:
with open(r"key.csv", 'w', newline='', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(['Target', 'Replacement'])
    writer.writerows(similar)

## Reading and Cleaning File

In [105]:
#Get simplist replacements for each target
keyDF = pd.read_csv(r"key.csv")

for item in range(len(keyDF)):
    for m in range(0, 50):
        for element in range(len(keyDF)):
            if keyDF['Target'][element] == keyDF['Replacement'][item]:
                # print("1. " + str(keyDF['Replacement'][item]))
                # print("2. " + str(keyDF['Target'][element]))
                keyDF['Replacement'][item] = keyDF['Replacement'][element]

lst = []
print(keyDF)
for i in range(len(keyDF)):
    temp = []
    temp.append(keyDF['Target'][i])
    temp.append(keyDF['Replacement'][i])
    lst.append(temp)
with open(r"key1.csv", 'w', newline='', encoding='UTF8') as f:
    # create the csv writer
    writer = csv.writer(f)
    writer.writerow(['Target', 'Replacement'])
    writer.writerows(lst)

                   Target Replacement
0                  ('y',)      ('x',)
1              ('z', 'y')      ('x',)
2         ('y', 'x', 'x')      ('x',)
3         ('h', 'y', 'h')      ('x',)
4         ('h', 'h', 'y')      ('x',)
..                    ...         ...
333  ('y', 'h', 'y', 'x')  ('x', 'h')
334  ('h', 'y', 'h', 'x')         NaN
335                ('z',)         NaN
336            ('y', 'y')         NaN
337       ('h', 'x', 'h')         NaN

[338 rows x 2 columns]
